In [0]:
dbutils.widgets.text("proc_date", "", "")
environment = dbutils.widgets.get("proc_date")

In [0]:
%run ../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
dbutils.widgets.text("proc_date", "", "")
proc_date = dbutils.widgets.get("proc_date")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {catalog_name}.udp_wcm_gold_db_logistic.a_aging_inventory (
  calday DATE,
  store_id STRING,
  product_id STRING,
  gr_date DATE,
  aging_days BIGINT,
  gr_qty FLOAT,
  gr_value FLOAT,
  gr_qty_running FLOAT,
  gr_value_running FLOAT,
  gr_qty_running_prev FLOAT,
  gr_value_running_prev FLOAT,
  bucket_id STRING,
  bucket_name STRING,
  closing_stock_quantity FLOAT,
  closing_stock_value FLOAT,
  flag BIGINT,
  aging_qty FLOAT,
  aging_value FLOAT,
  business_unit STRING,
  region STRING,
  is_win BIGINT,
  is_rural BIGINT,
  total_shelf_life FLOAT,
  origin STRING,
  assortment_type STRING,
  aging_cluster STRING,
  remaining_shelf_life_0 FLOAT,
  remaining_shelf_life_1 FLOAT,
  remaining_shelf_life_2 FLOAT
)
  TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
)
""")

In [0]:
from datetime import datetime, timedelta
import calendar


today_plus_7 = datetime.strptime(proc_date, "%Y-%m-%d") + timedelta(hours=7) # for back_dates
# today_plus_7 = datetime.utcnow() + timedelta(hours=7)

yesterday = today_plus_7 - timedelta(days=1)


last_day_of_month = datetime(yesterday.year, yesterday.month, calendar.monthrange(yesterday.year, yesterday.month)[1])


weekday = today_plus_7.weekday()  # Monday = 0
start_of_week = today_plus_7 - timedelta(days=weekday)
stock_date_week = start_of_week - timedelta(days=1)


if last_day_of_month.date() == yesterday.date() or stock_date_week.date() == yesterday.date():
    stock_date = yesterday.date()
else:
    stock_date = None

print("STOCK_DATE:", stock_date)

In [0]:
if f"{stock_date}" != None:

    spark.sql(f"""
    DELETE FROM {catalog_name}.udp_wcm_gold_db_logistic.a_aging_inventory
    WHERE CALDAY = '{stock_date}'
    """)
    spark.sql(f"""
    CREATE OR REPLACE TEMP VIEW aging_inventory_temp
    AS
    WITH STOCK (CALDAY, STORE_ID, PRODUCT_ID, CLOSING_STOCK_QUANTITY, CLOSING_STOCK_VALUE, STOCK_VALUE_UNIT) AS (
        SELECT
            a.CALDAY,
            a.STORE_ID,
            a.PRODUCT_ID,
            COALESCE(a1.CLOSING_STOCK_QUANTITY, 0) AS CLOSING_STOCK_QUANTITY,
            COALESCE(a1.CLOSING_STOCK_QUANTITY, 0) * COALESCE(a.CLOSING_STOCK_VALUE / NULLIF(a.CLOSING_STOCK_QUANTITY, 0), 0) AS CLOSING_STOCK_VALUE,
            GREATEST(COALESCE(a.CLOSING_STOCK_VALUE / NULLIF(a.CLOSING_STOCK_QUANTITY, 0), 0), 0) AS STOCK_VALUE_UNIT
        FROM {catalog_name}.udp_wcm_gold_vcm_dwh.f_inventory a
        LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.F_INVENTORY_SLOC_1000 a1
            ON a.CALDAY = a1.CALDAY AND a.STORE_ID = a1.STORE_ID AND a.PRODUCT_ID = a1.PRODUCT_ID
        LEFT JOIN  {catalog_name}.udp_wcm_gold_vcm_dwh.D_PRODUCT b
            ON a.PRODUCT_ID = b.PRODUCT_ID
        LEFT JOIN  {catalog_name}.udp_wcm_gold_vcm_dwh.D_STORE c
            ON a.STORE_ID = c.STORE_ID
        WHERE a.CALDAY =  '{stock_date}'
            AND b.MCH2_ID IN ('101', '102', '201', '202', '203')
            AND COALESCE(a1.CLOSING_STOCK_QUANTITY, 0) > 0.01
            AND c.BUSINESS_UNIT IN ('1500', '2000')
            AND COALESCE(b.PRODUCT_STATUS, '') <> 'Z9'
    ),
    MMIM03 (STORE_ID, PRODUCT_ID, CALDAY, MOVETYPE, DCINDIC, CPQUABU, STOCK_VALUE_UNIT) AS (
        SELECT  
            a.PLANT AS STORE_ID,
            a.MATERIAL AS PRODUCT_ID,
            a.CALDAY,
            a.MOVETYPE,
            a.DCINDIC,
            a.CPQUABU,
            COALESCE(d.STOCK_VALUE_UNIT, 0) AS STOCK_VALUE_UNIT
        FROM {catalog_name}.udp_wcm_silver_sap_bw.sap_vcdmmim03 a
        LEFT JOIN  {catalog_name}.udp_wcm_gold_vcm_dwh.D_PRODUCT b
            ON b.PRODUCT_ID = a.MATERIAL
        LEFT JOIN  {catalog_name}.udp_wcm_gold_vcm_dwh.D_STORE c
            ON c.STORE_ID = a.PLANT
        LEFT JOIN STOCK d
            ON d.STORE_ID = a.PLANT AND d.PRODUCT_ID = a.MATERIAL
        WHERE
            a.CALDAY > date_add( '{stock_date}', -720)
            AND a.CALDAY <=  '{stock_date}'
            AND a.MOVETYPE IN ('101', '102', '122', '123', '961', '962', '701', '702')
            AND a.STOCKTYPE = 'A'
            AND b.MCH2_ID IN ('101', '102', '201', '202', '203')
            AND c.BUSINESS_UNIT IN ('1500', '2000')
    ),
    MMIM03_GR (STORE_ID, PRODUCT_ID, CALDAY, GR_QTY, GR_VALUE) AS (
        SELECT
            STORE_ID,
            PRODUCT_ID,
            CALDAY,
            SUM(CASE WHEN DCINDIC = 'H' THEN -1 ELSE 1 END * ABS(CPQUABU)) AS GR_QTY,
            SUM(CASE WHEN DCINDIC = 'H' THEN -1 ELSE 1 END * ABS(CPQUABU) * STOCK_VALUE_UNIT) AS GR_VALUE
        FROM MMIM03
        GROUP BY STORE_ID, PRODUCT_ID, CALDAY
    ),
    STORE_GR (STORE_ID, PRODUCT_ID, GR_DATE, AGING_DAYS, GR_QTY, GR_VALUE, BUCKET_ID, BUCKET_NAME) AS (
        SELECT
            a.STORE_ID,
            a.PRODUCT_ID,
            a.CALDAY AS GR_DATE,
            datediff( '{stock_date}', a.CALDAY) AS AGING_DAYS,
            a.GR_QTY,
            a.GR_VALUE,
            b.BUCKET_ID,
            b.BUCKET_NAME
        FROM MMIM03_GR a
        LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_db_inventory.aging_bucket_fin_v2 b
            ON datediff( '{stock_date}', a.CALDAY) + 1 > b.LOWER_VALUE AND datediff( '{stock_date}', a.CALDAY) + 1 <= b.UPPER_VALUE
    ),
    STORE_GR_BUCKET (STORE_ID, PRODUCT_ID, GR_DATE, AGING_DAYS, BUCKET_ID, BUCKET_NAME, GR_QTY, GR_VALUE, GR_QTY_RUNNING, GR_VALUE_RUNNING) AS (
        SELECT
            STORE_ID,
            PRODUCT_ID,
            GR_DATE,
            AGING_DAYS,
            BUCKET_ID,
            BUCKET_NAME,
            GR_QTY,
            GR_VALUE,
            SUM(GR_QTY) OVER (PARTITION BY STORE_ID, PRODUCT_ID ORDER BY BUCKET_ID ASC, AGING_DAYS ASC) AS GR_QTY_RUNNING,
            SUM(GR_VALUE) OVER (PARTITION BY STORE_ID, PRODUCT_ID ORDER BY BUCKET_ID ASC, AGING_DAYS ASC) AS GR_VALUE_RUNNING
        FROM STORE_GR
        WHERE BUCKET_ID IS NOT NULL AND GR_QTY > 0
    ),
    STORE_GR_BUCKET_RUNNING_PREV (STORE_ID, PRODUCT_ID, GR_DATE, AGING_DAYS, BUCKET_ID, BUCKET_NAME, GR_QTY, GR_VALUE, GR_QTY_RUNNING, GR_VALUE_RUNNING, GR_QTY_RUNNING_PREV, GR_VALUE_RUNNING_PREV) AS (
        SELECT
            *,
            LAG(GR_QTY_RUNNING) OVER (PARTITION BY STORE_ID, PRODUCT_ID ORDER BY BUCKET_ID ASC, AGING_DAYS ASC) AS GR_QTY_RUNNING_PREV,
            LAG(GR_VALUE_RUNNING) OVER (PARTITION BY STORE_ID, PRODUCT_ID ORDER BY BUCKET_ID ASC, AGING_DAYS ASC) AS GR_VALUE_RUNNING_PREV
        FROM STORE_GR_BUCKET
    ),
    ALLOCATED_ROUND_ONE (CALDAY, STORE_ID, PRODUCT_ID, GR_DATE, AGING_DAYS, GR_QTY, GR_VALUE, GR_QTY_RUNNING, GR_VALUE_RUNNING, GR_QTY_RUNNING_PREV, GR_VALUE_RUNNING_PREV, BUCKET_ID, BUCKET_NAME, CLOSING_STOCK_QUANTITY, CLOSING_STOCK_VALUE, FLAG, AGING_QTY, AGING_VALUE) AS (
        SELECT
            a.CALDAY,
            a.STORE_ID,
            a.PRODUCT_ID,
            b.GR_DATE,
            COALESCE(b.AGING_DAYS, 721) AS AGING_DAYS,
            b.GR_QTY,
            b.GR_VALUE,
            b.GR_QTY_RUNNING,
            b.GR_VALUE_RUNNING,
            b.GR_QTY_RUNNING_PREV,
            b.GR_VALUE_RUNNING_PREV,
            COALESCE(b.BUCKET_ID, '7') AS BUCKET_ID,
            COALESCE(b.BUCKET_NAME, '> 720 ngày') AS BUCKET_NAME,
            a.CLOSING_STOCK_QUANTITY,
            a.CLOSING_STOCK_VALUE,
            1 AS FLAG,
            COALESCE(
                CASE
                    WHEN a.CLOSING_STOCK_QUANTITY > COALESCE(b.GR_QTY_RUNNING_PREV, 0) THEN LEAST(a.CLOSING_STOCK_QUANTITY - COALESCE(b.GR_QTY_RUNNING_PREV, 0), b.GR_QTY)
                    ELSE 0
                END,
                a.CLOSING_STOCK_QUANTITY
            ) AS AGING_QTY,
            COALESCE(
                CASE
                    WHEN a.CLOSING_STOCK_VALUE > COALESCE(b.GR_VALUE_RUNNING_PREV, 0) AND a.CLOSING_STOCK_VALUE >= 0 THEN
                        LEAST(a.CLOSING_STOCK_VALUE - COALESCE(b.GR_VALUE_RUNNING_PREV, 0), b.GR_VALUE)
                    WHEN a.CLOSING_STOCK_VALUE < 0 THEN a.CLOSING_STOCK_VALUE
                    ELSE 0
                END,
                a.CLOSING_STOCK_VALUE
            ) AS AGING_VALUE
        FROM STOCK a
        LEFT JOIN STORE_GR_BUCKET_RUNNING_PREV b
        ON a.STORE_ID = b.STORE_ID AND a.PRODUCT_ID = b.PRODUCT_ID
        where not (round(COALESCE(
                CASE
                    WHEN a.CLOSING_STOCK_QUANTITY > COALESCE(b.GR_QTY_RUNNING_PREV, 0) THEN LEAST(a.CLOSING_STOCK_QUANTITY - COALESCE(b.GR_QTY_RUNNING_PREV, 0), b.GR_QTY)
                    ELSE 0
                END,
                a.CLOSING_STOCK_QUANTITY
            ),3) <= 0.001
            AND  
                round( COALESCE(
                CASE
                    WHEN a.CLOSING_STOCK_VALUE > COALESCE(b.GR_VALUE_RUNNING_PREV, 0) AND a.CLOSING_STOCK_VALUE >= 0 THEN
                        LEAST(a.CLOSING_STOCK_VALUE - COALESCE(b.GR_VALUE_RUNNING_PREV, 0), b.GR_VALUE)
                    WHEN a.CLOSING_STOCK_VALUE < 0 THEN a.CLOSING_STOCK_VALUE
                    ELSE 0
                END,
                a.CLOSING_STOCK_VALUE
            ),3) <= 0.001)
   
    ),
    ALLOCATED_ROUND_TWO (CALDAY, STORE_ID, PRODUCT_ID, GR_DATE, AGING_DAYS, GR_QTY, GR_VALUE, GR_QTY_RUNNING, GR_VALUE_RUNNING, GR_QTY_RUNNING_PREV, GR_VALUE_RUNNING_PREV, BUCKET_ID, BUCKET_NAME, CLOSING_STOCK_QUANTITY, CLOSING_STOCK_VALUE, FLAG, AGING_QTY, AGING_VALUE) AS
   
    (
        SELECT * FROM ALLOCATED_ROUND_ONE
        UNION ALL
        SELECT
            CALDAY,
            STORE_ID,
            PRODUCT_ID,
            NULL AS GR_DATE,
            721 AS AGING_DAYS,
            NULL AS GR_QTY,
            NULL AS GR_VALUE,
            NULL AS GR_QTY_RUNNING,
            NULL AS GR_VALUE_RUNNING,
            NULL AS GR_QTY_RUNNING_PREV,
            NULL AS GR_VALUE_RUNNING_PREV,
            '7' AS BUCKET_ID,
            '> 720 ngày' AS BUCKET_NAME,
            CLOSING_STOCK_QUANTITY,
            CLOSING_STOCK_VALUE,
            2 AS FLAG,
            CLOSING_STOCK_QUANTITY - AGING_QTY_SUM AS AGING_QTY,
            CLOSING_STOCK_VALUE - AGING_VALUE_SUM AS AGING_VALUE
        FROM (
            SELECT
                *,
                SUM(AGING_QTY) OVER(PARTITION BY STORE_ID, PRODUCT_ID) AS AGING_QTY_SUM,
                SUM(AGING_VALUE) OVER(PARTITION BY STORE_ID, PRODUCT_ID) AS AGING_VALUE_SUM
            FROM ALLOCATED_ROUND_ONE
            QUALIFY
                ROUND(CLOSING_STOCK_QUANTITY, 3) > COALESCE(ROUND(SUM(AGING_QTY) OVER(PARTITION BY STORE_ID, PRODUCT_ID), 3), 0)
                AND ROW_NUMBER() OVER (PARTITION BY STORE_ID, PRODUCT_ID ORDER BY BUCKET_ID DESC, AGING_DAYS DESC, GR_QTY ASC) = 1
        )
        WHERE NOT((ROUND(AGING_QTY, 3) <= 0.001) AND (ROUND(AGING_VALUE, 3) <= 0.001))
    ),
    ALLOCATED_ROUND_THREE (CALDAY, STORE_ID, PRODUCT_ID, GR_DATE, AGING_DAYS, GR_QTY, GR_VALUE, GR_QTY_RUNNING, GR_VALUE_RUNNING, GR_QTY_RUNNING_PREV, GR_VALUE_RUNNING_PREV, BUCKET_ID, BUCKET_NAME, CLOSING_STOCK_QUANTITY, CLOSING_STOCK_VALUE, FLAG, AGING_QTY, AGING_VALUE) AS (
        SELECT * FROM ALLOCATED_ROUND_TWO
        UNION ALL
        SELECT
            CALDAY,
            STORE_ID,
            PRODUCT_ID,
            NULL AS GR_DATE,
            721 AS AGING_DAYS,
            NULL AS GR_QTY,
            NULL AS GR_VALUE,
            NULL AS GR_QTY_RUNNING,
            NULL AS GR_VALUE_RUNNING,
            NULL AS GR_QTY_RUNNING_PREV,
            NULL AS GR_VALUE_RUNNING_PREV,
            '7' AS BUCKET_ID,
            '> 720 ngày' AS BUCKET_NAME,
            CLOSING_STOCK_QUANTITY,
            CLOSING_STOCK_VALUE,
            3 AS FLAG,
            0 AS AGING_QTY,
            CLOSING_STOCK_VALUE - AGING_VALUE_SUM AS AGING_VALUE
        FROM
            (
                SELECT
                    *,
                    SUM(AGING_VALUE) OVER(PARTITION BY STORE_ID, PRODUCT_ID) AS AGING_VALUE_SUM
                FROM ALLOCATED_ROUND_TWO
                QUALIFY ROUND(CLOSING_STOCK_VALUE, 3) > COALESCE(ROUND(SUM(AGING_VALUE) OVER(PARTITION BY STORE_ID, PRODUCT_ID), 3), 0)
                AND ROW_NUMBER() OVER (PARTITION BY STORE_ID, PRODUCT_ID ORDER BY BUCKET_ID DESC, AGING_DAYS DESC) = 1
            )
        WHERE ROUND(CLOSING_STOCK_VALUE - AGING_VALUE_SUM, 3) > 0.001
    ),
    AGING_DATA (CALDAY, STORE_ID, PRODUCT_ID, GR_DATE, AGING_DAYS, GR_QTY, GR_VALUE, GR_QTY_RUNNING, GR_VALUE_RUNNING, GR_QTY_RUNNING_PREV, GR_VALUE_RUNNING_PREV, BUCKET_ID, BUCKET_NAME, CLOSING_STOCK_QUANTITY, CLOSING_STOCK_VALUE, FLAG, AGING_QTY, AGING_VALUE, BUSINESS_UNIT, REGION, IS_WIN, IS_RURAL, TOTAL_SHELF_LIFE, ORIGIN, ASSORTMENT_TYPE) AS (
        SELECT
            a.*,
            b.BUSINESS_UNIT,
            b.REGION_DOMAIN_VN AS REGION,
            CASE WHEN UPPER(b.CONCEPT) = 'WIN' THEN 1 ELSE 0 END AS IS_WIN,
            CASE WHEN UPPER(b.CONCEPT) = 'RURAL' THEN 1 ELSE 0 END AS IS_RURAL,
            c.TOTAL_SHELF_LIFE,
            c.ORIGIN,
            d.CORE_RANGE AS ASSORTMENT_TYPE
        FROM ALLOCATED_ROUND_THREE a
        LEFT JOIN  {catalog_name}.udp_wcm_gold_vcm_dwh.D_STORE b
            ON a.STORE_ID = b.STORE_ID
        LEFT JOIN  {catalog_name}.udp_wcm_gold_vcm_dwh.D_PRODUCT c
            ON a.PRODUCT_ID = c.PRODUCT_ID
        LEFT JOIN   {catalog_name}.udp_wcm_gold_vcm_dwh.D_CORE_RANGE_ASSORTMENT d
            ON a.STORE_ID = d.STORE_ID AND a.PRODUCT_ID = d.PRODUCT_ID
    ),
    AGING_CLUSTER_REGION (BUSINESS_UNIT, PRODUCT_ID, REGION, AGING_CLUSTER) AS (
        SELECT
            BUSINESS_UNIT,
            PRODUCT_ID,
            REGION,
            CASE
                WHEN COUNT(DISTINCT CASE WHEN BUCKET_ID >= '5' THEN STORE_ID ELSE NULL END) = 0 THEN 'No'
                WHEN COUNT(DISTINCT CASE WHEN BUCKET_ID >= '5' THEN STORE_ID ELSE NULL END) > 0 AND COUNT(DISTINCT CASE WHEN BUCKET_ID >= '5' THEN STORE_ID ELSE NULL END) < 0.33 * COUNT(DISTINCT STORE_ID) THEN 'Local'
                WHEN COUNT(DISTINCT CASE WHEN BUCKET_ID >= '5' THEN STORE_ID ELSE NULL END) > 0.67 * COUNT(DISTINCT STORE_ID) THEN 'Regional'
                ELSE 'Local - Regional'
            END AS AGING_CLUSTER
        FROM AGING_DATA
        GROUP BY BUSINESS_UNIT, PRODUCT_ID, REGION
    ),
    AGING_CLUSTER (CALDAY, STORE_ID, PRODUCT_ID, GR_DATE, AGING_DAYS, GR_QTY, GR_VALUE, GR_QTY_RUNNING, GR_VALUE_RUNNING, GR_QTY_RUNNING_PREV, GR_VALUE_RUNNING_PREV, BUCKET_ID, BUCKET_NAME, CLOSING_STOCK_QUANTITY, CLOSING_STOCK_VALUE, FLAG, AGING_QTY, AGING_VALUE, BUSINESS_UNIT, REGION, IS_WIN, IS_RURAL, TOTAL_SHELF_LIFE, ORIGIN, ASSORTMENT_TYPE,
   
     AGING_CLUSTER) AS (
        SELECT
            a1.*,
            CASE
                WHEN a2.AGING_CLUSTER = 'Local' AND BUCKET_ID >= '5' THEN 'Local'
                WHEN a2.AGING_CLUSTER = 'Regional' AND BUCKET_ID >= '5' THEN 'Regional'
                WHEN a2.AGING_CLUSTER = 'Local - Regional' AND BUCKET_ID >= '5' THEN 'Local - Regional'
                ELSE 'No'
            END AS AGING_CLUSTER
        FROM AGING_DATA a1
        LEFT JOIN AGING_CLUSTER_REGION a2
            ON a1.BUSINESS_UNIT = a2.BUSINESS_UNIT AND a1.PRODUCT_ID = a2.PRODUCT_ID AND a1.REGION = a2.REGION
    ),
    REMAINING_SHELF_LIFE (CALDAY, STORE_ID, PRODUCT_ID, GR_DATE, AGING_DAYS, GR_QTY, GR_VALUE, GR_QTY_RUNNING, GR_VALUE_RUNNING, GR_QTY_RUNNING_PREV, GR_VALUE_RUNNING_PREV, BUCKET_ID, BUCKET_NAME, CLOSING_STOCK_QUANTITY, CLOSING_STOCK_VALUE, FLAG, AGING_QTY, AGING_VALUE, BUSINESS_UNIT, REGION, IS_WIN, IS_RURAL, TOTAL_SHELF_LIFE, ORIGIN, ASSORTMENT_TYPE, AGING_CLUSTER, REMAINING_SHELF_LIFE_0, REMAINING_SHELF_LIFE_1,
     REMAINING_SHELF_LIFE_2) AS (
        SELECT
            *,
            GREATEST(CASE WHEN ORIGIN = 'VN' THEN TOTAL_SHELF_LIFE * 0.75 - AGING_DAYS ELSE TOTAL_SHELF_LIFE * 0.63 - AGING_DAYS END, 0) AS REMAINING_SHELF_LIFE_0,
            GREATEST(CASE WHEN ORIGIN = 'VN' THEN TOTAL_SHELF_LIFE * 0.83 - AGING_DAYS ELSE TOTAL_SHELF_LIFE * 0.75 - AGING_DAYS END, 0) AS REMAINING_SHELF_LIFE_1,
            GREATEST(CASE WHEN ORIGIN = 'VN' THEN TOTAL_SHELF_LIFE * 0.88 - AGING_DAYS ELSE TOTAL_SHELF_LIFE * 0.81 - AGING_DAYS END, 0) AS REMAINING_SHELF_LIFE_2
        FROM AGING_CLUSTER
    ),
    MAIN AS (
        SELECT
            *
        FROM REMAINING_SHELF_LIFE
    )
    SELECT
    CALDAY,
    STORE_ID,
    PRODUCT_ID,
    GR_DATE,
    AGING_DAYS,
    GR_QTY,
    GR_VALUE,
    GR_QTY_RUNNING,
    GR_VALUE_RUNNING,
    GR_QTY_RUNNING_PREV,
    GR_VALUE_RUNNING_PREV,
    BUCKET_ID,
    BUCKET_NAME,
    CLOSING_STOCK_QUANTITY,
    CLOSING_STOCK_VALUE,
    FLAG,
    AGING_QTY,
    AGING_VALUE,
    BUSINESS_UNIT,
    REGION,
    IS_WIN,
    IS_RURAL,
    TOTAL_SHELF_LIFE,
    ORIGIN,
    ASSORTMENT_TYPE,
    AGING_CLUSTER,
    REMAINING_SHELF_LIFE_0,
    REMAINING_SHELF_LIFE_1,
    REMAINING_SHELF_LIFE_2
    FROM MAIN;
    """)
 
    spark.sql(f"""
    INSERT INTO  {catalog_name}.udp_wcm_gold_db_logistic.a_aging_inventory
    (
    CALDAY,
    STORE_ID,
    PRODUCT_ID,
    GR_DATE,
    AGING_DAYS,
    GR_QTY,
    GR_VALUE,
    GR_QTY_RUNNING,
    GR_VALUE_RUNNING,
    GR_QTY_RUNNING_PREV,
    GR_VALUE_RUNNING_PREV,
    BUCKET_ID,
    BUCKET_NAME,
    CLOSING_STOCK_QUANTITY,
    CLOSING_STOCK_VALUE,
    FLAG,
    AGING_QTY,
    AGING_VALUE,
    BUSINESS_UNIT,
    REGION,
    IS_WIN,
    IS_RURAL,
    TOTAL_SHELF_LIFE,
    ORIGIN,
    ASSORTMENT_TYPE,
    AGING_CLUSTER,
    REMAINING_SHELF_LIFE_0,
    REMAINING_SHELF_LIFE_1,
    REMAINING_SHELF_LIFE_2
    )
    SELECT
    CALDAY,
    STORE_ID,
    PRODUCT_ID,
    GR_DATE,
    AGING_DAYS,
    GR_QTY,
    GR_VALUE,
    GR_QTY_RUNNING,
    GR_VALUE_RUNNING,
    GR_QTY_RUNNING_PREV,
    GR_VALUE_RUNNING_PREV,
    BUCKET_ID,
    BUCKET_NAME,
    CLOSING_STOCK_QUANTITY,
    CLOSING_STOCK_VALUE,
    FLAG,
    AGING_QTY,
    AGING_VALUE,
    BUSINESS_UNIT,
    REGION,
    IS_WIN,
    IS_RURAL,
    TOTAL_SHELF_LIFE,
    ORIGIN,
    ASSORTMENT_TYPE,
    AGING_CLUSTER,
    REMAINING_SHELF_LIFE_0,
    REMAINING_SHELF_LIFE_1,
    REMAINING_SHELF_LIFE_2
    FROM aging_inventory_temp
    """)
 